In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI


chat = OpenAI(
    temperature=0.1,
    max_tokens=450,
    model="gpt-3.5-turbo"
)

chat.save("model.json")